# Sign Language Classification

Даны  картинки с символами языка жестов ASL (Americal Sign Language). Необходимо построить модель машинного обучения, которая смогла бы правильно распознавать символы языка жестов и классифицировать их в определенные группы. 

![](https://user-images.githubusercontent.com/34737471/55292500-52acb580-5409-11e9-9a78-7cdc0d8da8b6.png)

## 1. Первичный анализ данных

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [5]:
train_df.head()

In [7]:
train_df.shape, test_df.shape

In [6]:
y_train = train_df['label']
y_test = test_df['label']

X_train = train_df.drop(columns='label')
X_test = test_df.drop(columns='label')

In [7]:
def show_images(images,labels):
    images = images.values.reshape(-1,28,28,1)
    unique_labels = y_train.unique()
    fig,ax = plt.subplots(2,5)
    fig.set_size_inches(10, 6)
    k =0
    for i in range(2):
        for j in range(5):
            ax[i,j].imshow(images[k] , cmap='gray')
            ax[i,j].set_title(str(unique_labels[y_train[k]]))
            k = k+1;
    plt.tight_layout()

In [8]:
show_images(X_train, y_train)

Распределение классов в trainset

In [9]:
plt.figure(figsize=(15,6))
sns.set_style("darkgrid")
sns.countplot(y_train)
plt.show()

In [10]:
plt.figure(figsize=(15,6))
sns.set_style("darkgrid")
sns.countplot(y_test)
plt.show()

## 2. Построение простых методов многоклассовой классификации

Использованные модели:

* Линейные модели
* Решающие деревья
* Байесовские методы
* Нейронные сети
* Сверточные нейронные сети

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from xgboost.sklearn import XGBClassifier 
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [21]:
svc = SVC(random_state=5)
svc.fit(X_train, y_train)
print('='*25)
print('SVC')
print(f'accuracy of train set: {svc.score(X_train, y_train)}')
print(f'accuracy of test set: {svc.score(X_test, y_test)}')

In [22]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
print('='*25)
print('LinearSVC')
print(f'accuracy of train set: {lsvc.score(X_train, y_train)}')
print(f'accuracy of test set: {lsvc.score(X_test, y_test)}')

In [24]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
print('='*25)
print('SGD Classifier')
print(f'accuracy of train set: {sgd.score(X_train, y_train)}')
print(f'accuracy of test set: {sgd.score(X_test, y_test)}')

In [43]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_train, y_train)}')
print(f'accuracy of test set: {knn.score(X_test, y_test)}')

In [27]:
rf = RandomForestClassifier(random_state=5)
rf.fit(X_train, y_train)
print('='*25)
print('RandomForestClassifier')
print(f'accuracy of train set: {rf.score(X_train, y_train)}')
print(f'accuracy of test set: {rf.score(X_test, y_test)}')

In [32]:
decisiontree = DecisionTreeClassifier(max_depth=50, random_state=5)
decisiontree.fit(X_train, y_train)
print('='*25)
print('DecisionTreeClassifier')
print(f'accuracy of train set: {decisiontree.score(X_train, y_train)}')
print(f'accuracy of test set: {decisiontree.score(X_test, y_test)}')

In [45]:
bagging = BaggingClassifier(ExtraTreeClassifier(random_state=5))
bagging.fit(X_train, y_train)
print('='*25)
print('BaggingClassifier')
print(f'accuracy of train set: {bagging.score(X_train, y_train)}')
print(f'accuracy of test set: {bagging.score(X_test, y_test)}')

In [36]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
print('='*25)
print('LGBM Classifier')
print(f'accuracy of train set: {lgbm.score(X_train, y_train)}')
print(f'accuracy of test set: {lgbm.score(X_test, y_test)}')

In [42]:
catboost = CatBoostClassifier(learning_rate=0.3, max_depth=10, n_estimators=25)
catboost.fit(X_train, y_train)
print('='*25)
print('CatBoost Classifier')
print(f'accuracy of train set: {catboost.score(X_train, y_train)}')
print(f'accuracy of test set: {catboost.score(X_test, y_test)}')

In [46]:
%time
estimators = [
    ("svc", svc), 
    ("knn", knn),
    ("rf", rf),
    ("extratree", bagging),
    ("lgbm", lgbm)]

vote = VotingClassifier(estimators=estimators)
vote.fit(X_train, y_train)
print('='*25)
print('VotingClassifier')
print(f'accuracy of train set: {vote.score(X_train, y_train)}')
print(f'accuracy of test set: {vote.score(X_test, y_test)}')

In [95]:
y_pred = vote.predict(X_test)

In [100]:
print(classification_report(y_test, y_pred))

In [97]:
plt.figure(figsize=(15,6))
cf_matrix = confusion_matrix(y_test, y_pred)

In [99]:
sns.heatmap(cf_matrix, annot=True)
plt.show()

PCA, TSNE

In [10]:
y_train = train_df['label']
y_test = test_df['label']

X_train = train_df.drop(columns='label')
X_test = test_df.drop(columns='label')

In [11]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [1]:
%%time
only_pca_model = PCA()
only_pca_model.n_components = 200
only_pca_data = only_pca_model.fit_transform(X/255)
only_pca_data = np.vstack((only_pca_data.T)).T

pca_tsne_model = TSNE(n_components=3, random_state=0, perplexity=50, n_iter=1000)
pca_tsne_data = pca_tsne_model.fit_transform(only_pca_data)


pca_tsne_data = np.vstack((pca_tsne_data.T, y)).T
pca_tsne_train = pd.DataFrame(pca_tsne_data)
pca_tsne_train.head()

In [15]:
X_pca_train = pca_tsne_train.drop(columns=['label'])[:27455]
y_pca_train = pca_tsne_train['label'][:27455]

X_pca_test = pca_tsne_train.drop(columns=['label'])[27455:]
y_pca_test = pca_tsne_train['label'][27455:]

In [16]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_pca_train, y_pca_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_pca_train, y_pca_train)}')
print(f'accuracy of test set: {knn.score(X_pca_test, y_pca_test)}')

In [102]:
sns.FacetGrid(pca_tsne_train, hue='label', height=8).map(plt.scatter, 'PC 1', "PC 2").add_legend()
plt.show()

In [103]:
X_pca_train = pca_tsne_train.drop(columns=['label'])[:27455]
y_pca_train = pca_tsne_train['label'][:27455]

X_pca_test = pca_tsne_train.drop(columns=['label'])[27455:]
y_pca_test = pca_tsne_train['label'][27455:]

In [108]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_pca_train, y_pca_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_pca_train, y_pca_train)}')
print(f'accuracy of test set: {knn.score(X_pca_test, y_pca_test)}')

In [ ]:
не правильно, подсматриваю в y

In [89]:
only_pca_model = decomposition.PCA()
only_pca_model.n_components = 200
only_pca_data = only_pca_model.fit_transform(X/255)
only_pca_data = np.vstack((only_pca_data.T, y)).T

pca_tsne_model = TSNE(n_components =2, random_state =0, perplexity =50, n_iter=1000, verbose=1)
pca_tsne_data = model.fit_transform(only_pca_data)


pca_tsne_data = np.vstack((pca_tsne_data.T, y)).T
pca_tsne_train = pd.DataFrame(pca_tsne_data, columns = ('PC 1', 'PC 2', 'label'))
pca_tsne_train.head()

In [90]:
sns.FacetGrid(pca_tsne_train, hue='label', height=8).map(plt.scatter, 'PC 1', "PC 2").add_legend()
plt.show()

In [92]:
y_train.shape

In [93]:
X_pca_train = pca_tsne_train.drop(columns=['label'])[:27455]
y_pca_train = pca_tsne_train['label'][:27455]

X_pca_test = pca_tsne_train.drop(columns=['label'])[27455:]
y_pca_test = pca_tsne_train['label'][27455:]

In [94]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_pca_train, y_pca_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_pca_train, y_pca_train)}')
print(f'accuracy of test set: {knn.score(X_pca_test, y_pca_test)}')

## 3. Построение нейронных сетей

In [ ]:
import torch
import torchvision

import torch.nn as nn

In [ ]:
X = torch.FloatTensor(X_train.values)
y = torch.LongTensor(y_train.values) # CrossEntropyLoss требует формата int64

In [ ]:
X_train.shape

In [ ]:
X

In [ ]:
network = torch.nn.Sequential(
    torch.nn.Linear(784, 128), 
    torch.nn.Sigmoid(), 
    torch.nn.Linear(128, 64), 
    torch.nn.Sigmoid(),
    torch.nn.Linear(64, 25) 
)

optimizer = torch.optim.SGD(network.parameters(), lr=0.1, weight_decay=0.1)
criterion = torch.nn.CrossEntropyLoss() 

for i in range(1000):
    optimizer.zero_grad()
    pred = network(X)
#     print(pred.shape, y.shape)
    loss = criterion(pred,y.squeeze(0))
    loss.backward()
    optimizer.step()
    if i%100==0:
        print(loss.item())

In [ ]:
res = torch.softmax(pred, 1)
print(res)

In [ ]:
res[0]

In [ ]:
res[1]

In [ ]:
res[55]

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_df, 
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_df, 
                                          batch_size=100, 
                                          shuffle=False)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        output_1 = self.relu(self.layer_1(inputs))
        output = self.layer_2(output_1)
        
        return output

In [ ]:
model = NeuralNet(28*28, 500, 25)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
len(train_loader)

In [ ]:
enumerate(train_loader)

In [ ]:
train_loader

In [ ]:
# test_df = pd.read_csv('../input/mnist-in-csv/mnist_test.csv')
# train_df = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')

train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_labels = train_df['label'].values # converting to numpy also

test_labels=test_df['label'].values
train_images = (train_df.iloc[:,1:].values).astype('float32')
test_images = (test_df.iloc[:,1:].values).astype('float32')

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28)
test_images = test_images.reshape(test_images.shape[0], 28, 28)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
train_images_tensor = torch.tensor(train_images)/255.0 #default torch.FloatTensor
train_labels_tensor = torch.tensor(train_labels)
train_tensor = TensorDataset(train_images_tensor, train_labels_tensor)

test_images_tensor = torch.tensor(test_images)/255.0
test_labels_tensor = torch.tensor(test_labels)
test_tensor = TensorDataset(test_images_tensor, test_labels_tensor)

In [ ]:
# train_loader = DataLoader(train_tensor, batch_size=16, num_workers=2, shuffle=True)
# test_loader = DataLoader(test_images_tensor, batch_size=16, num_workers=2, shuffle=False)

In [ ]:
train_loader = DataLoader(train_tensor, batch_size=16, num_workers=2, shuffle=True)
test_loader = DataLoader(test_images_tensor, batch_size=16, num_workers=2, shuffle=False)

In [ ]:
torch.manual_seed(0)

total_step = len(train_loader)
for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

In [ ]:
# На этапе предсказания не требуется вычислять градиенты:
pred_lst = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader): 
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        pred_lst.extend(list(predicted))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = model

In [ ]:
evaluate_x.shape

In [ ]:
evaluate_y.shape

In [ ]:
if (device.type=='cuda'):
    evaluate_x=test_images_tensor.cuda()
    evaluate_y=test_labels_tensor.cuda()
else:
    evaluate_x=test_images_tensor.reshape(-1, 28*28)
    evaluate_y=test_labels_tensor
    

output = net(evaluate_x)

pred = output.data.max(1)[1]
d = pred.eq(evaluate_y.data).cpu()
a=(d.sum().data.cpu().numpy())
b=d.size()
b=torch.tensor(b)
b=(b.sum().data.cpu().numpy())
accuracy = a/b

print('Accuracy:', accuracy*100)

In [ ]:
y_test = test_df['label']
y_test

In [ ]:
len(pred)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
#Print confusion matrix
cm = confusion_matrix(y_test, pred)

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
sns.set(font_scale=1.6)
sns.heatmap(cm, annot=True, linewidths=.5, ax=ax)

Построение CNN